## Veri Setini İçe Aktarma

In [18]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
veriSeti = pd.read_csv("caesarian.csv")
df = veriSeti.copy()
df.head()

,Yas,Kacinci Dogum,Dogum Zamani,Kan Basinci,Kalp Problemi,Sezeryan
0,22,1,0,2,0,0
1,26,2,0,1,0,1
2,26,2,1,1,0,0
3,28,1,0,2,0,0
4,22,2,0,1,0,1


## Ön İşleme, Encoding

In [19]:
from sklearn.preprocessing import OneHotEncoder
oneHot = OneHotEncoder(handle_unknown='ignore')

oheDogumZamani = oneHot.fit_transform(df[' Dogum Zamani'].values.reshape(-1,1)).toarray()
dfdz = pd.DataFrame(oheDogumZamani, columns=['Zamaninda Dogum', 'Erken Dogum', 'Gec Dogum'])

oheKanBasinci = oneHot.fit_transform(df[' Kan Basinci'].values.reshape(-1,1)).toarray()
dfkb = pd.DataFrame(oheKanBasinci, columns=['Dusuk KB', 'Normal KB', 'Yuksek KB'])

oheKalpProblemi = oneHot.fit_transform(df[' Kalp Problemi'].values.reshape(-1,1)).toarray()
dfkp = pd.DataFrame(oheKalpProblemi, columns=['Kalp Problemi Yok', 'Kalp Problemi Var'])

dfMain = df.join(dfdz)
dfMain = dfMain.join(dfkb)
dfMain = dfMain.join(dfkp)

dfMain.drop(' Dogum Zamani', axis=1, inplace=True)
dfMain.drop(' Kan Basinci', axis=1, inplace=True)
dfMain.drop(' Kalp Problemi', axis=1, inplace=True)

dfMain.head()

,Yas,Kacinci Dogum,Sezeryan,Zamaninda Dogum,Erken Dogum,Gec Dogum,Dusuk KB,Normal KB,Yuksek KB,Kalp Problemi Yok,Kalp Problemi Var
0,22,1,0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,26,2,1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,26,2,0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
3,28,1,0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,22,2,1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


## Test ve Eğitim Verilerini Ayırma

In [20]:
from sklearn.model_selection import train_test_split
x = dfMain.drop(' Sezeryan', axis=1)
y = dfMain[' Sezeryan']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.22, random_state=88)

## Ölçeklendirme

In [21]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

## Modelleme - Logistic Regression

In [22]:
from sklearn.linear_model import LogisticRegression
modelLR = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=250,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=50, solver='warn', tol=0.00005, verbose=0,
                   warm_start=False)
modelLR.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=250,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=50, solver='warn', tol=5e-05, verbose=0,
                   warm_start=False)

## Modelleme - Naive Bayes

In [23]:
from sklearn.naive_bayes import GaussianNB
modelNB = GaussianNB(priors=None, var_smoothing=1e-10)
modelNB.fit(x_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-10)

## Tahmin Aşaması - Logistic Regression

In [24]:
y_predLR = modelLR.predict(x_test)
print(y_predLR)

[0 1 1 0 1 1 1 0 1 0 1 1 0 0 1 0 1 1]


In [25]:
print(y_test.values)

[0 1 1 0 1 1 1 1 1 0 1 1 0 0 1 0 1 1]


## Tahmin Aşaması - Naive Bayes

In [26]:
y_predNB = modelNB.predict(x_test)
print(y_predNB)

[0 1 0 0 1 1 1 0 1 0 1 1 0 0 1 0 1 1]


In [27]:
print(y_test.values)

[0 1 1 0 1 1 1 1 1 0 1 1 0 0 1 0 1 1]


## Performans Kriteri - Logistic Regression

In [28]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
print(confusion_matrix(y_test, y_predLR))

[[ 6  0]
 [ 1 11]]


In [29]:
print(classification_report(y_test, y_predLR))

              precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
           1       1.00      0.92      0.96        12

    accuracy                           0.94        18
   macro avg       0.93      0.96      0.94        18
weighted avg       0.95      0.94      0.95        18



In [30]:
accuracy_score(y_test, y_predLR)

0.9444444444444444

## Performans Kriteri - Naive Bayes

In [31]:
print(confusion_matrix(y_test, y_predNB))

[[ 6  0]
 [ 2 10]]


In [32]:
print(classification_report(y_test, y_predNB))

              precision    recall  f1-score   support

           0       0.75      1.00      0.86         6
           1       1.00      0.83      0.91        12

    accuracy                           0.89        18
   macro avg       0.88      0.92      0.88        18
weighted avg       0.92      0.89      0.89        18



In [33]:
accuracy_score(y_test, y_predNB)

0.8888888888888888

## Sonuç

In [34]:
print("Logistic Regression Doğruluk Oranı: %", "%.2f" % (100 * accuracy_score(y_test, y_predLR)))
print("Naive Bayes Doğruluk Oranı: %", "%.2f" % (100 * accuracy_score(y_test, y_predNB)))

Logistic Regression Doğruluk Oranı: % 94.44
Naive Bayes Doğruluk Oranı: % 88.89
